 #                                         CAPSTONE PROJECT

# Predicting Mental Health in Tech workplace

In [1]:
# importting reguired libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from warnings import filterwarnings
filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [2]:
pd.options.display.max_columns=None

In [3]:
# importing dataset
df = pd.read_csv('survey.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'survey.csv'

## DATA UNDERSTANDING

### Columns in dataset:
**•Timestamp**<br>
**• Age**<br>
**• Gender**<br>
**• Country**<br>
**• state** : If you live in the United States, which state or territory do you live in?<br>
**• self_employed** : Are you self-employed?<br>
**• family_history** : Do you have a family history of mental illness?<br>
**• treatment** : Have you sought treatment for a mental health condition?<br>
**• work_interfere** : If you have a mental health condition, do you feel that it interferes with your work?<br>
**• no_employees** : How many employees does your company or organization have?<br>
**• remote_work** : Do you work remotely (outside of an office) at least 50% of the time?<br>
**• tech_company** : Is your employer primarily a tech company/organization?<br>
**• benefits** : Does your employer provide mental health benefits?<br>
**• care_options** : Do you know the options for mental health care your employer provides?<br>
**• wellness_program** : Has your employer ever discussed mental health as part of an employee wellness program?<br>
**• seek_help** : Does your employer provide resources to learn more about mental health issues and how to seek help?<br>
**• anonymity** : Is your anonymity protected if you choose to take advantage of mental health or substance abuse treat<br>
**• leave** : How easy is it for you to take medical leave for a mental health condition?<br>
**• mentalhealthconsequence** : Do you think that discussing a mental health issue with your employer would have negative consequences<br>
**• physhealthconsequence** : Do you think that discussing a physical health issue with your employer would have negative consequences<br>
**• coworkers** : Would you be willing to discuss a mental health issue with your coworkers?<br>
**• supervisor** : Would you be willing to discuss a mental health issue with your direct supervisor(s)?<br>
**• mentalhealthinterview** : Would you bring up a mental health issue with a potential employer in an interview? <br>
**• physhealthinterview** : Would you bring up a physical health issue with a potential employer in an interview?<br>
**• mentalvsphysical** : Do you feel that your employer takes mental health as seriously as physical health?<br>
**• obs_consequence** : Have you heard of or observed negative consequences for coworkerswith mental health conditions<br>
**• comments**    


In [ ]:
# information about the dataset
df.info()

There 2 columns in total, out of we have only are numeric datatype and remaining 26 columns are object data type.<br>
Out of the 26 object datatype columns, 24 columns are catergorical variables.

In [ ]:
# Description about the numeric column
df.describe().applymap('{:,.2f}'.format)

There are 1259 entries in the age column.The maximun and minimum age values doesnot make sense. Hence we have to check for garbage values in the column

In [ ]:
# Description of object type variables
df.describe(include='object')

The column Gender has 49 unique entries. We will look into that further.<br>
The few columns seems to have null values. Depending upon the null value percentage of each column and the value it adds to the dataset, we take appropritae actions further.

### Check for null values

In [ ]:
# Percentage of null values in each column
((df.isnull().sum()/len(df))*100).sort_values(ascending=False)

In [ ]:
# plotting the null values in heatmap
plt.figure(figsize=(14,7))
sns.heatmap(df.isnull(),cmap='Purples')
plt.show()

The column comments has comments 87% null values. Hence we drop the column.<br>
The column state has 41% null values and it is not adding significant information to the dataset. Hence we drop the column.<br>
The column work interfere has 21 % null values. This column adds significant information, hence we replace the null values with the mode of the column.<br>
The column self employed has only 1.4% missing value. This also replaced with mode values.

### Check for duplicate values

In [ ]:
df.duplicated().sum()

There are no duplicated values present in the dataset

## DATA PREPROCESSING

Here we perform preprocession to drop the redundant columns and clean the data from garbage values

In [ ]:
# Droping the columns timestamp, state, comments
df.drop(columns=['Timestamp', 'state', 'comments'],axis=1,inplace=True)

In [ ]:
df.head(2)

Cleaning the data from garbage values

Checking the column Age

In [ ]:
plt.figure(figsize=(12,6))
sns.kdeplot(x=df['Age'])
plt.show()

There are age values greate than 100 and negative values, which is not possible. Removing the age values greater than 80 and less than 13.(As the official age to work part time in  UK and USA is 13 and 14 respectively).

In [ ]:
age_mismatch  = df[(df['Age']>80) | (df['Age']<13)].index

In [ ]:
df.drop(index=age_mismatch,axis=0,inplace=True)

The column Gender has 49 unique values. For the sake of simplicity we categorise them into Male, Female and Others.

In [ ]:
df.shape

In [ ]:
df.Gender.unique()

In [ ]:
df['Gender'] = df['Gender'].replace({'male':'Male','M':'Male','m':'Male','Make':'Male','Man':'Male','Malr':'Male',
                'Male (CIS)':'Male','Male ':'Male','Cis Male':'Male','Mail':'Male','Cis Male':'Male','cis male':'Male',
                'msle':'Male','Cis Man':'Male','maile':'Male','Mal':'Male',
                'female':'Female','F':'Female','f':'Female','Woman':'Female','Femake':'Female','Female ':'Female',
                'femail':'Female', 'Female (cis)':'Female','woman':'Female','cis-female/femme':'Female','Cis Female':'Female',
                'Female (trans)':'Others','Androgyne':'Others','queer':'Others','Neuter':'Others',
                'Trans woman':'Others','fluid':'Others','non-binary':'Others','queer/she/they':'Others','Enby':'Others',
                'Trans-female':'Female','Genderqueer':'Others','Agender':'Others','something kinda male?':'Others','Male-ish':'Others','ostensibly male, unsure what that really means':'Others',
                'Nah':'Others','Guy (-ish) ^_^':'Others','male leaning androgynous':'Others','A little about you':'Others'})

In [ ]:
df.Gender.unique()

## EXPLORATORY DATA ANALYSIS

### Univariant Analysis

In [ ]:
df.columns

### Age

In [ ]:
plt.subplots(1,2,figsize=(20,8))

plt.subplot(121)
sns.distplot(x=df['Age'])
plt.xlabel('Age')


plt.subplot(122)
sns.histplot(x=df.Age,binwidth=10)

plt.suptitle ('Distribution of Age',fontsize=20)
plt.show()

Majority employees age is in their 30's. The column age is right skewed. 

### Gender

In [ ]:
df.Gender.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x=df['Gender'])
plt.title('Count plot of Gender',fontsize=15)
plt.show()

Majority of them employees are male. Other gender group is the minority genger 

### Country

In [ ]:
df.Country.value_counts()

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x=df['Country'])
plt.xticks(rotation=90)
plt.title('Distribution of employees wrt to countries',fontsize=15)
plt.show()

In [ ]:
df.Country.nunique()

The are employees from 46 countries who have taken the survey. 50% of the people are from United states. There are also significant employees from United Kingdom and Canada.

### Self employed

In [ ]:
df.self_employed.value_counts()

In [ ]:
sns.countplot(x=df['self_employed'])
plt.title('Are you self-employeed?',fontsize=15)
plt.show()

Majority (80%) of the people work for others

### Family history

In [ ]:
df.family_history.value_counts()

In [ ]:
sns.countplot(x=df['family_history'])
plt.title(' Any family history of mental illness?',fontsize=15)
plt.show()

60% of the employees have no family history mental illness

###  Treatment

In [ ]:
df.treatment.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.treatment.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Have sought treatment for a mental health condition?',fontsize=15)
plt.show()

50% employeses Have sought treatment for a mental health condition

###  Work interfere

In [ ]:
df.work_interfere.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.work_interfere.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Interference of Mental health condition on Work?',fontsize=15)
plt.show()

78 % of employees have experienced some sort of work interference due to mental illness

### No_employees

In [ ]:
df.no_employees.value_counts()

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x=df.no_employees)
plt.title('No. of employees in company/organisation',fontsize=15)
plt.show()

Most people work in a small employerss group(<100) and followed by large group(>1000) employees

### Remote work

In [ ]:
df.remote_work.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.remote_work.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Work outside office',fontsize=15)
plt.show()

 Only 30% of people are able to work remotely

### Tech company

In [ ]:
df.tech_company.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.tech_company.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Primarily a tech company/organization?',fontsize=15)
plt.show()

82% of employer is primarily a tech company/organization

### Benefits

In [ ]:
df.benefits.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.benefits.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Does employer provide mental health benefits?',fontsize=15)
plt.show()

38% of employees have been provided with mental health benifits 
32% of employeers don't know about the mental helath benifits provided by the employeers. Which shows the lack of awarness

### Care options

In [ ]:
df.care_options.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.care_options.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Know about the options provided for mental health care?',fontsize=15)
plt.show()

Only 35% of employees know about the options for mental health care their employer provides.
This shows lack of awarness amoung employees

### Wellness program

In [ ]:
df.wellness_program.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.wellness_program.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title(' Have discussed mental health as part of wellness program?',fontsize=15)
plt.show()

67% of employer have not discussed mental health as part of an employee wellness program.
 Mental health is not given as much importance as physical health by employess

###  Seek help

In [ ]:
df.seek_help.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.seek_help.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Provide resources to learn more about mental health issues and how to seek help?',fontsize=15)
plt.show()

Only 20% of employers provide resources to learn more about mental health issues and how to seek help

### Anonymity 

In [ ]:
df.anonymity.value_counts()

In [ ]:
plt.subplots(1,2,figsize=(14,6))

plt.subplot(121)
df.anonymity.value_counts().plot(kind='pie',autopct='%0.2f%%')

plt.subplot(122)
sns.countplot(x=df.anonymity)

plt.suptitle('Is anonymity protected ? ', fontsize=15)
plt.show()

65% of employees are unaware about anaonymity protection provided by employers
Out of aware employees, majority of the employers provide anonymity protection

### Leave 

In [ ]:
df.leave.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.leave.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Ease of leave for mental health condition',fontsize=15)
plt.show()

45% of the employees are ignorant about the easy of leave to take leave for medical health conditions
Out of the known employees, majority of employees have ease in applying leave for mental health conditions

### Mental Health Consequence
       

In [ ]:
df.mental_health_consequence.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.mental_health_consequence.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Does discussing a mental health issue with employerhave negative consequences ?',fontsize=15)
plt.show()

39% of employees think that discussing a mental health issue with your employer would not have negative consequences
But 38% of employees  think that discussing a mental health issue with your employer would have negative consequences which shows the stigma around mental health

### Physical Health Consequence

In [ ]:
df.phys_health_consequence.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.phys_health_consequence.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Does discussing a physical health issue with employerhave negative consequences ?',fontsize=15)
plt.show()

74% of employees think that discussing a physical health issue with your employer would have no negative consequences
This shows that majority of employees have no problem in discussing physical health with employers

###  Coworkers 

In [ ]:
df.coworkers.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.coworkers.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Willing to discuss a mental health issue with coworkers?',fontsize=15)
plt.show()

Only 18 % of the employees are willing to discuss a mental health issue with coworkers
61.5 % of the employees are willing to discuss a mental health issue with some of the coworkers meaning that they are willing to openly dicuss it with everyone

### Supervisor
       

In [ ]:
df.supervisor.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.supervisor.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Willing to discuss a mental health issue with direct supervisor?',fontsize=15)
plt.show()

41% of employees are willing to discuss a mental health issue with your direct supervisor(s), which is a good sign
31% of employees are not willing to discuss a mental health issue with your direct supervisor(s)

### Mental Health Interview

In [ ]:
df.mental_health_interview.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.mental_health_interview.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Talk about mental health issue in an interview?',fontsize=15)
plt.show()

80% of employees are not willing to bring up a mental health issue with a potential employer in an interview.
Only 3%  of employees are willing to bring up a mental health issue with a potential employer in an interview.
16.5% of employees are open to bring up a mental health issue with a potential employer in an interview.
This show the stigma around mental health.

###  Physical Health Interview
       

In [ ]:
df.phys_health_interview.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.phys_health_interview.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Talk about physical health issue in an interview?',fontsize=15)
plt.show()

Only 40% of employees are not willing to bring up a physical health issue with a potential employer in an interview which is half of the people are not willing to bring up a mental health issue with a potential employer in an interview.
16% of employees are willing to bring up a physical health issue with a potential employer in an interview which is 5 times that of mental health issues interview.
44% of employees are open to bring up a physical health issue with a potential employer in an interview.
This shows discrimination between physical and mental health issues in employees view.

### Mental vs Physical 

In [ ]:
df.mental_vs_physical.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.mental_vs_physical.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Is mental health as seriously as physical health?',fontsize=15)
plt.show()

46 % of employees don't know about the opinion of employer.Remaining employees have  uniform opinion abouthow their employers takes mental health as seriously as physical health. 

### Observed Consequence

In [ ]:
df.obs_consequence.value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df.obs_consequence.value_counts().plot(kind='pie',autopct='%0.2f%%')
plt.title('Heard of or observed negative consequences for coworkers with mental health conditions',fontsize=15)
plt.show()

85.5% of employees have not heard of or observed negative consequences for coworkers with mental health conditions.
Shows that mental health of a person is not affecting coworkers.

### Bivariant Analysis

### Age Vs Gender

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df.Gender,y=df.Age)
plt.title('Variation of Age wrt Gender',fontsize=15)
plt.show()

The  age of male employees has higher median , we can say that on average age of male employee is higher.<br> 
Minumum age of female n nale employee facing health issue is 20


In [ ]:
plt.figure(figsize=(14,7))
sns.boxplot(x=df.Country,y=df.Age)
plt.xticks(rotation=75)
plt.title('Distribution of Age wrt Country',fontsize=15)
plt.show()

Employees with age >50 are from USA. USA has the maximum age interval of employees. This may be due to a more number of responses from USA.

### Age Vs Self Employed

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df.self_employed,y=df.Age)
plt.title('Variation of Age wrt employement status',fontsize=15)
plt.show()

The age of self-employed employees has higher median , we can say that on average age of self-employed employee is higher<br>Age of self-employed Employees ranges from 20 to 60 above.<br>Age of on self-employed Employees ranges 20 to 60

### Age Vs Family History

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df['family_history'],y=df['Age'])
plt.title('Variation of Age wrt Family history',fontsize=15)
plt.show()

The age of employees with  family history 0f mental health is slighly  higher median ,thus we can say that average age of employee having no family history id 35.
The overall range for the employess with no family history is higher but the IQR has lower values.  

### Age Vs Treatment

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df.treatment,y=df.Age)
plt.title('Variation of Age wrt Treatment',fontsize=15)
plt.show()

The average age of  employees  who have sought their treatment is slighly higher.<br>
Age of employee who have not sought their treatment can be above 60

### Age Vs Work_interfere

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df.work_interfere,y=df.Age)
plt.title('Variation of Age wrt Work Interference',fontsize=15)
plt.show()

Employees who has never faced work interference are of slightly lower age 

### Age Vs No of Employees

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(x=df.no_employees,y=df.Age)
plt.title('Variation of Age wrt Employees',fontsize=15)
plt.show()

The average age of  employees in organization having more than 1000 employees is  higher.<br>
Maximun age of employee in organization having more than 1000 employees ranges upto 60.<br>
Maximun age of employee in organization having 6-25 employees ranges above 60

### Age Vs Remote work

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df.remote_work,y=df.Age)
plt.title('Variation of Age wrt Remote work',fontsize=15)
plt.show()

The average age of  employees  work remotely (outside of an office) is slighly higher.<br>
Age of employees who are allowed to work remotely are above

### Age Vs Tech Company

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df.tech_company,y=df.Age)
plt.title('Variation of Age wrt Tech company',fontsize=15)
plt.show()

On average the employees whose employers are not  a tech company/organization  have high age <br>
Minimum age of employee whose employers are not  a tech company/organization is 20 

### Age Vs Benefits

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df.benefits,y=df.Age)
plt.title('Variation of Age wrt recognition of Benfits',fontsize=15)
plt.show()

On an average the age of employees who know about the mental helath benifits provided by the employeer is higher.<br> 
The centre of distribution Yes is the highest of the three distributions (median is 32).The distribution  is negatively skewed because the whisker and half-box are longer on the left side of the median than on the right side.<br>
Majority of employes above age 55 know about the mental helath benifits provided by the employeer is higher.


### Age Vs Care options

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df.care_options,y=df.Age)
plt.title('Variation of Age wrt Care options',fontsize=15)
plt.show()

On an average the age of employees who know about the mental helath care options provided by the employeer is higher.<br>
The centre of distribution Yes is the highest of the three distributions (median is 32).The distribution  is negatively skewed because the whisker and half-box are longer on the left side of the median than on the right side.<br>
Distribution of age of employees who are not sure and dont know about careoptions  is approximately symmetric, because both half-boxes are almost the same.<br> 
Age employees who dont know about gealth care options ranges above 60.

### Age Vs Wellness_program

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df.wellness_program,y=df.Age)
plt.title('Variation of Age wrt Wellness program',fontsize=15)
plt.show()

On an average the age of employees whose employer  discussed mental health as part of an employee wellness program is higher.<br>
The centre of distribution of age of employyes whose employer  discussed mental health is the highest of the three distributions (median is 32).The distribution  is negatively skewed because the whisker and half-box are longer on the left side of the median than on the right side.<br>
There are employees above age 48 whose employes did not discuss with them about mental health as part of wellnes sprogram

### Age Vs Seek help

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df.seek_help,y=df.Age)
plt.title('Variation of Age wrt Seek help',fontsize=15)
plt.show()

Average age of employees whose employer provide resources to learn more about mental health issues are higher .
<br>Few employees above age 50 dont know about the resources that employer provide to learn about health issue and how to seek help.
<br>Majority of employees of age above 48 are those whose employer does not  provide resources to learn more about mental health issues are higher .

### Age Vs Anonymity

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df.anonymity,y=df.Age)
plt.title('Variation of Age wrt Anonymity',fontsize=15)
plt.show()

In [ ]:
#majority of employee above age 48 dont know about anonymity


### Age Vs Leave

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(x=df.leave,y=df.Age)
plt.show()

### Age Vs Mental_health_consequence

In [ ]:
sns.boxplot(x=df.mental_health_consequence,y=df.Age)
plt.show()

Average age of employee who think think that discussing a mental health issue with your employer would have negative consequences is high.
Majority of employee above age 50 think that it might be possible to have negative consequences with employer if they discuss about mental health 

### Age Vs Physical_health_consequence

In [ ]:
sns.boxplot(x=df.phys_health_consequence,y=df.Age)
plt.show()

Majority of employees about age 48 think think that discussing about physical health issue with your employer would not  have negative consequences is high

### Age Vs Coworkers

In [ ]:
sns.boxplot(x=df.coworkers,y=df.Age)
plt.show()

Majotity of employees above age 50 will only be comforatable with some of the workers to discuss about mental health

### Age Vs Supervisor

In [ ]:
sns.boxplot(x=df.supervisor,y=df.Age)
plt.show()

Some of the employees's age who are willing to discuss a mental health issue with  direct supervisor has highest average age.
Majority of employees above age 50 are willing to discuss a mental health issue with  direct supervisor

### Age Vs Mental_health_interview

In [ ]:
sns.boxplot(x=df.mental_health_interview,y=df.Age)
plt.show()

Employees above age 50 will not bring up a mental health issue with a potential employer in an interview.
Distribution employees age who are ready to bring up a mental health issue with a potential employer in an interviewis approximately symmetric, because both half-boxes are almost the same 


### Age Vs Physical_health_interview

In [ ]:
sns.boxplot(x=df.phys_health_interview,y=df.Age)
plt.show()

Average age of employees who are not willing to bring up a physical health issue with a potential employer in an interview is higher.
Age of employees who are not willing to bring up a physical health issue with a potential employer in an interview ranges above 60


### Age Vs Mental_health and Physical_health

In [ ]:
sns.boxplot(x=df.mental_vs_physical,y=df.Age)
plt.show()

Average of empoyer who does not takes mental health as seriously as physical health is higher .
Majority of employers above age 48 takes mental health as seriously as physical health

### Age Vs Obesity_consequence

In [ ]:
sns.boxplot(x=df.obs_consequence,y=df.Age)
plt.show()

Majority of employees above age 50 did not  observed negative consequences for coworkers with mental health conditions in your workplace

In [ ]:
df.columns

In [ ]:
df.head()

### Gender Vs Self_employed

In [ ]:
pd.crosstab(df['self_employed'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Majority of males and females are not self employed.

In [ ]:
df.info()

### Gender Vs Family_history

In [ ]:
pd.crosstab(df['family_history'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Majority of males have no family history of mental health issues.
Slightly higher no. of females and other genders have family history of mental health issues.

### Gender Vs Treatment

In [ ]:
pd.crosstab(df['treatment'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Higher no. of males are not undergoing any treatment regarding their mental health.
Higher no of females and other  genders are undergoing any treatment regarding their mental health.

### Gender Vs Work_intrefere

In [ ]:
pd.crosstab(df['work_interfere'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

All the genders are facing some kind of work interference

### Gender Vs Remote_work

In [ ]:
pd.crosstab(df['remote_work'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Similar distribution can be seen in all genders. We may conclude that working remotely is independent of gender.

### Gender Vs Tech_company

In [ ]:
pd.crosstab(df['tech_company'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Majority of the males are working in tech company/organization.
The proportion of females working in tech companies is less than males.

### Gender Vs Benefits

In [ ]:
pd.crosstab(df['benefits'],df['Gender']).plot(kind='bar',stacked=True,figsize=(12,6))
plt.show()

Mental health benifits proportion is higher females.
<br>Ratio of males who are unaware of about the mental health benifits provided by the employer is higher.
<br>Mental health benifits provided by the employer may be gender depended.

### Gender Vs Care_options

In [ ]:
pd.crosstab(df['care_options'],df['Gender']).plot(kind='bar',figsize=(12,6),stacked=True)
plt.show()

Majority of the males don't know about the health care_options provided by employeer.
Comparatively higher proportion of females known about health care_options.

### Gender Vs Wellness_program

In [ ]:
pd.crosstab(df['wellness_program'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Similary distribution can be seen in all genders wrt wellness program. Hence we may conclude that employer discussion
<br>on mental health as part of an employee wellness program is independent of gender.

### Gender Vs Seek_help

In [ ]:
pd.crosstab(df['seek_help'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Similary distribution can be seen in all genders wrt seek help. Hence we may conclude that employer providing resources to learn more about mental health issues and how to seek help is independent of gender.

### Gender Vs Anonymity

In [ ]:
pd.crosstab(df['anonymity'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Similary distribution can be seen in all genders wrt anonymity. Hence we may conclude that anonymity protection
<br>if you choose to take advantage of mental health or substance abuse treat is independent of gender.

### Gender Vs Leave

In [ ]:
pd.crosstab(df['leave'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

There is no much difference in distribution of gender wrt to leave.
We may say the easy of take medical leave for a mental health condition is independent of gender

### Gender Vs Mental_health_consequence

In [ ]:
pd.crosstab(df['mental_health_consequence'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Higher no. of males think that discussing a mental health issue with your employer would not have negative consequences
<br>Higher no. of females think that discussing a mental health issue with your employer might have negative consequences

### Gender Vs Physical_health_consequence

In [ ]:
pd.crosstab(df['phys_health_consequence'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

In all genders, majority of the employees think that discussing a physical health issue with your employer would not have negative consequences

### Gender Vs Coworkers

In [ ]:
pd.crosstab(df['coworkers'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

All the gender employeers are mostly willing to discuss a mental health issue only with some of the coworkers.

### Gender Vs Supervisor

In [ ]:
pd.crosstab(df['supervisor'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Majority of males are willing to discuss a mental health issue with direct supervisor(s)
<br>Proportion of females willing to discuss a mental health issue with direct supervisor(s) is less
<br>There may be gender dependency on willingness to discuss a mental health issue with direct supervisor(s)

### Gender Vs Mental_health_interview

In [ ]:
pd.crosstab(df['mental_health_interview'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Major no. of employees of all genders are not willing to bring up a mental health issue with a potential employer in an interview
<br>Specifically, the proportion of females are not willing to bring up a mental health issue with a potential employer
<br>in an interview,compared to other genders
<br>We may say that willingness to bring up a mental health issue with a potential employer in an interview is genders dependent.

### Gender Vs Physical_health_interview

In [ ]:
pd.crosstab(df['phys_health_interview'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Majority of males are open to discuss physical health issue with a potential employer in an interview
<br>Maximum no. of females are not willing to discuss physical health issue with a potential employer in an interview
<br>Minimum no. of employees of all genders are willing to discuss physical health issue with a potential employer in an interview.
<br>we may say that the willingness to discuss physical health issue with a potential employer in an interview id dependent on gender

### Gender Vs Mental_vs_Physical

In [ ]:
pd.crosstab(df['mental_vs_physical'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Maximum no. of males and females have don't know if their employer takes mental health as seriously as physical health.
<br> Amoung the employees who have an opinion, a slightly higher no.females feel that their employer takes mental health 
as seriously as physical health
<br>Amoung the employees who have an opinion,50% males feel that their employer takes mental health as seriously as 
physical health and remaining 50% males feels the opposite.

### Gender Vs Obeserved_consequence

In [ ]:
pd.crosstab(df['obs_consequence'],df['Gender']).plot(kind='bar',figsize=(12,6))
plt.show()

Majority of employees of all genders haven't heard of or observed negative consequences for coworkers with 
mental health conditions

### family_history vs treatment

In [ ]:
pd.crosstab(df["family_history"],df['treatment']).plot(kind='bar',figsize=(12,6))
plt.show()

Employees with  no family history are less likely to get treament when compared to the employees with family history
<br> Majority of employees with significant margin don't have family history of health issues

### family_history vs work_interfere

In [ ]:
pd.crosstab(df["family_history"],df['work_interfere']).plot(kind='bar',figsize=(12,6))
plt.show()

Employees with family history have their work interfered oftentimes or some times more than employees with out family history
<br>Employees without family history are more likely to don't have their work interfered ever 

### family_history vs mental_health_consequence

In [ ]:
pd.crosstab(df["family_history"],df['mental_health_consequence']).plot(kind='bar',figsize=(12,6))
plt.show()

A higher proportion of employees with no family history that discussing a mental health issue with your employer 
would not have negative consequences.
<br>Unlike employees with family history, a higher proportion think that discussing a mental health issue with your employer 
might have negative consequences

### family_history vs phys_health_consequence

In [ ]:
pd.crosstab(df["family_history"],df['phys_health_consequence']).plot(kind='bar',figsize=(12,6))
plt.show()

discussing a physical health issue with your employer is not affect by family history 

### family_history vs coworkers

In [ ]:
pd.crosstab(df['coworkers'],df["family_history"]).plot(kind='bar',figsize=(12,6))
plt.show()

willing to discuss a mental health issue with coworkers is independent of employees family history

### family_history vs supervisor

In [ ]:
pd.crosstab(df["family_history"],df['supervisor']).plot(kind='bar',figsize=(12,6))
plt.show()

willingness to discuss a mental health issue with direct supervisor(s) is independent of family history of employee

### family_history vs mental_health_interview

In [ ]:
pd.crosstab(df['mental_health_interview'],df["family_history"]).plot(kind='bar',figsize=(12,6))
plt.show()

bringing up a mental health issue with a potential employer in an interview is independent of family history

### family_history vs phys_health_interview

In [ ]:
pd.crosstab(df['phys_health_interview'],df["family_history"]).plot(kind='bar',figsize=(12,6))
plt.show()

bringing up a physical health issue with a potential employer in an interview is independent of family history

### family_history vs mental_vs_physical

In [ ]:
pd.crosstab(df['mental_vs_physical'],df["family_history"]).plot(kind='bar',figsize=(12,6))
plt.show()

minimum no. employees with family history feel that your employer takes mental health as seriously as physical health

### treatment vs work_interfere

In [ ]:
pd.crosstab(df['work_interfere'],df['treatment']).plot(kind='bar',figsize=(12,6))
plt.show()

Employees who sought treatment for a mental health condition, find some kind of work interference because of mental health condition

### treatment vs remote_work

In [ ]:
pd.crosstab(df['remote_work'],df['treatment']).plot(kind='bar',figsize=(12,6))
plt.show()

Majority of employees are not working remotely.
<br>Amoung the employees working remotely, slightly higher no. of them have sought treatment for a mental health condition

### treatment vs mental_health_consequence

In [ ]:
pd.crosstab(df['mental_health_consequence'],df['treatment']).plot(kind='bar',figsize=(12,6))
plt.show()

Employees who sought treatment for mental health condition think that discussing a mental health issue with your employer 
would have negative consequences than employees who have not sought treatment

### treatment vs phys_health_consequence

In [ ]:
pd.crosstab(df['phys_health_consequence'],df['treatment']).plot(kind='bar',figsize=(12,6))
plt.show()

Majority of employees who have not sought or sought treatment think that discussing a physical health issue with your 
employer wouldnot have negative consequences
<br>Amoung the employees who thing think that discussing a physical health issue with your employer would have negative
consequences, ratio  of employees who have sought treatment are higher

### treatment vs coworkers

In [ ]:
pd.crosstab(df['coworkers'],df['treatment']).plot(kind='bar',figsize=(12,6))
plt.show()

Employees who have sought their treatment are willing to discuss a mental health issue with your coworkers as compared to who have not sought their treatment.
<br> both employees who have sought or not sought treatment are willing to discuss a mental health issue with your coworkers

### treatment vs leave

In [ ]:
pd.crosstab(df['treatment'],df['leave']).plot(kind='bar',figsize=(12,6))
plt.show()

Majority of employees who have not sought treatment dont know about leaves provided for mental health condition
<br> More no. Employees who have sought their treatment found somewhat or very difficult to apply medical leave for mental health condition

### treatment vs mental_health_interview

In [ ]:
pd.crosstab(df['treatment'],df['mental_health_interview']).plot(kind='bar',figsize=(12,6))
plt.show()

Majority of employees who have sought their treatment are not willing to bring up a mental health issue with a potential employer in an interview
<br>Employees who have  not sought their treatment might not bring up a mental health issue with a potential employer in an interview

### treatment vs supervisor

In [ ]:
pd.crosstab(df['treatment'],df['supervisor']).plot(kind='bar',figsize=(12,6))
plt.show()

Majority of employyes who have not sought their treatment are willing to discuss a mental health issue with your direct supervisor
<br>Majority of employyes who have sought their treatment are not willing to discuss a mental health issue with your direct supervisor

### treatment vs phys_health_interview

In [ ]:
pd.crosstab(df['treatment'],df['phys_health_interview']).plot(kind='bar',figsize=(12,6))
plt.show()

Employees who sought treatment for mental health conditions are not willing to discuss physical health issue with a
potential employer in an interview

### treatment vs mental_vs_physical

In [ ]:
pd.crosstab(df['mental_vs_physical'],df['treatment']).plot(kind='bar',figsize=(12,6))
plt.show()

Higher proportion of employees who have sought treatment think that their employer donot takes mental health as seriously as physical health

### remote_work vs benefits

In [ ]:
pd.crosstab(df['benefits'],df['remote_work']).plot(kind='bar',figsize=(12,6))
plt.show()

from above we say that the people who does not do remote work were high in number compare to people who does the remote work.
people who does not do remote work were more aware about the benifits.
<br>there is around 30% people who does the remote work knows about benifits.
<br>so we can say that people who does not do remote work were more aware about the benifits than others.

### remote_work vs mental_vs_physical

In [ ]:
pd.crosstab(df['remote_work'],df['mental_vs_physical']).plot(kind='bar',figsize=(12,6))
plt.show()

For employees who work remotely, think that their employer takes mental health as seriously as physical health.
which is not the case for non-remote type of work.

### remote_work vs mental_health_consequence

In [ ]:
pd.crosstab(df['remote_work'],df['mental_health_consequence']).plot(kind='bar',figsize=(12,6))
plt.show()

Employees who work remotely, think that discussing a mental health issue with your employer would not have 
negative consequences unlike employees who work non-remote. 

### remote_work vs phys_health_consequence

In [ ]:
pd.crosstab(df['remote_work'],df['phys_health_consequence']).plot(kind='bar',figsize=(12,6))
plt.show()

Employees opinion on discussing a physical health issue with your employer having negative consequences is not affected by remote work.

### remote_work vs mental_health_interview


In [ ]:
pd.crosstab(df['remote_work'],df['mental_health_interview']).plot(kind='bar',figsize=(12,6))
plt.show()

Openess to bring up a mental health issue with a potential employer in an interview is independent of work type(remote or not ).

### remote_work vs phys_health_interview

In [ ]:
pd.crosstab(df['remote_work'],df['phys_health_interview']).plot(kind='bar',figsize=(12,6))
plt.show()

Openess to bring up a physical health issue with a potential employer in an interview is independent of work type(remote or not ).

In [ ]:
df.columns

### Self employed vs remote work

In [ ]:
pd.crosstab(df['self_employed'],df['remote_work']).plot(kind='bar',figsize=(12,6))
plt.show()

Higher ratio of people who are self employeed are working remotely
<br>from above we can say that people who are not self employed are not working remotely.

### Self employeed vs mental health consequences

In [ ]:
pd.crosstab(df['self_employed'],df['mental_health_consequence']).plot(kind='bar',figsize=(12,6))
plt.show()

Higher proportion of people who are self employed,think that discussing a mental health issue with your employer wouldnot
have negative consequences

### Self employeed vs physical health consequences

In [ ]:
pd.crosstab(df['self_employed'],df['phys_health_consequence']).plot(kind='bar',figsize=(12,6))
plt.show()

Discussing a physical health issue with your employer is independent of employement type(Self or not)

### Self employeed vs physical health interview

In [ ]:
pd.crosstab(df['self_employed'],df['phys_health_interview']).plot(kind='bar',figsize=(12,6))
plt.show()

openess bring up a mental health issue with a potential employer in an interview is independent of employement type(Self or not)

### Self employeed vs mental health interview

In [ ]:
pd.crosstab(df['self_employed'],df['mental_health_interview']).plot(kind='bar',figsize=(12,6))
plt.show()

Maximum no. of people are not willing to u bring up a physical health issue with a potential employer in an interview 
irrespective of employement type(self or not)

### Self employeed vs Mental vs Physical

In [ ]:
pd.crosstab(df['self_employed'],df['mental_vs_physical']).plot(kind='bar',figsize=(12,6))
plt.show()

A higher proportion of people who are self employeed feel that your employer takes mental health as seriously as physical health.
Which is not the case for people who are not self employeed.

### co workers vs mental health consequences

In [ ]:
pd.crosstab(df['coworkers'],df['mental_health_consequence']).plot(kind='bar',figsize=(12,6))
plt.show()

Maximum no. of employees willing to discuss a mental health issue with coworkers are not willing bring up a 
mental health issue with a potential employer in an interview

### Coworkers vs Physical health consequences

In [ ]:
pd.crosstab(df['coworkers'],df['phys_health_consequence']).plot(kind='bar',figsize=(12,6))
plt.show()

Irrespective of willingness to discuss a mental health issue with coworkers, employees opinion on discussing a 
physical health issue with employer is the same.

### Coworkers vs Phy_health_interview

In [ ]:
pd.crosstab(df['phys_health_interview'],df['coworkers']).plot(kind='bar',figsize=(12,6))
plt.show()

The ratio of employees willinging to discuss a mental health issue with coworkers and employees willinging to bring up a 
physical health issue with a potential employer in an interview is slightly higher.

### Coworker vs mental health interview

In [ ]:
pd.crosstab(df['mental_health_interview'],df['coworkers']).plot(kind='bar',figsize=(12,6))
plt.show()

Higher ratio of employees who willinging to discuss a mental health issue with coworkers are also willinging to bring up  
physical health issue with a potential employer in an interview.

## Multi Variant Analysis

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(x=df['Gender'],y=df['Age'],hue=df['family_history'])
plt.title('Correation between Age, Gender and Family history',fontsize=15)
plt.show()

The average age of females who have family history is less than the average age of females who have family history.
The average age of males and other gender who have family history is greater than the average age of females who have family history.

### Does gender and family history have effect on treatment ?

In [ ]:
pd.pivot_table(df,index =['treatment'],columns=['Gender','family_history'],values=['Age'], aggfunc='count',margins=True)

Higher number of females with family history are undergoing treatement for mrntal health condition.Highest number of males with no family history are not undergoing treatment.
Looks like gender and family history may have effect on treatment.

In [ ]:
pd.pivot_table(df,index =['treatment'],columns=['Gender','work_interfere'],values=['Age'], aggfunc='count',margins=True)

A very small count of employees of all genders who are undergoing treatment, feel that their mental health condition never interferes with work.
Most of the employees of all genders who are undergoing treatment, have experienced some sort of work interference. 

### OUTLIERS TREATMENT 

In [ ]:
sns.kdeplot(x=df['Age'])
plt.title('Kde plot of age before transformation')
plt.show()

There are outliers present in the variable age. Reducing the skewness by using boxcox transformation.

In [ ]:
# Q1 = df['Age'].quantile(0.25)
# Q3 = df['Age'].quantile(0.75)

# IQR = Q3-Q1
# UL = Q3+1.5*IQR
# LL = Q1-1.5*IQR

# age_out = df[(df['Age']<LL)|(df['Age']>UL)].index
# df.drop(index=age_out,axis=0,inplace=True)

In [ ]:
box_age = stats.boxcox(df['Age'])
sns.kdeplot(box_age[0])
plt.title('Kde plot of age after using boxcox transformation')
plt.show()

In [ ]:
print('The skewness of age after transformation',stats.skew(box_age[0]))

The age curve looks almost normal after boxcox transformation and the skewness of the age has been reduced to 0.0005 which is almost normal distribution 

In [ ]:
df['Age'] = box_age[0]

# Missing Value

In [ ]:
null_df = pd.DataFrame()
null_df['Count'] = df.isnull().sum()
null_df['Percent'] = null_df['Count']/len(null_df)
null_df[null_df['Percent']!=0].sort_values('Percent',ascending=False)

In [ ]:
plt.figure(figsize=(14,7))
sns.heatmap(df.isnull(),cmap='Purples')
plt.show()

### Imputing missing values with mode 

In [ ]:
df.self_employed.value_counts()

In [ ]:
wi_freq_value = df.work_interfere.mode() 
df['work_interfere'].fillna(wi_freq_value[0],inplace=True)

In [ ]:
df['self_employed'].fillna(df['self_employed'].mode()[0],inplace=True)

In [ ]:
df.isnull().sum().sum()

In [ ]:
# No missing values

### Statistical test to see if Mental health issues is related to country

In [ ]:
# Considering only US, UK and Canada countries to analysis the effect of country on mental health

In [ ]:
df_country = df[(df['Country']=='United States') |( df['Country']=='United Kingdom')|( df['Country']=='Canada')]

In [ ]:
df_country.shape

In [ ]:
from scipy.stats import chi2_contingency

Hypothesis<br>
H0: Mental health treatment is indepedent on country<br>
H1: Mental health traetment is dependent on country

In [ ]:
country_MH_table = pd.crosstab(df_country['Country'],df_country['treatment'])
country_MH_table

In [ ]:
obs_val = country_MH_table.values
alpha = 0.01
test_stat,p,dof,exp_val = chi2_contingency(obs_val)
print('p-value :',p)
print('Test Statistics :',test_stat)

Here p-value is greater than 0.01. Hence we fail to reject null hypothesis and we can counclude that<br>
Mental health issue treatment is independed on country.<br>
(Note: Considering only US, UK and Canada)

But here we are droping country column as the no. of entries in the each country is not sufficient and it effects the model(most of the countries have only 1 entry).  This affects the model significantly. 

In [ ]:
df.drop('Country',axis=1,inplace=True)

### Statistical test to see if Mental health issues is related to gender

Hypothesis<br>
H0: Mental health issue is indepedent on gender<br>
H1: Mental health issue is dependent on gender

In [ ]:
gender_MH_table = pd.crosstab(df['Gender'],df['treatment'])
gender_MH_table

In [ ]:
obs_val = gender_MH_table.values
alpha = 0.05
test_stat,p,dof,exp_val = chi2_contingency(obs_val)
print('p-value :',p)
print('Test Statistics :',test_stat)

Here p-value is less than 0.05. Hence we reject null hypothesis and we can counclude that Mental health issue treatment is depended on gender.

### Statistical test to see if Mental health treatment has impact on work interference.

Hypothesis<br>
H0: Mental health treatment is indepedent on work interference<br>
H1: Mental health treatment is dependent on work interference

In [ ]:
work_int_table = pd.crosstab(df['work_interfere'],df['treatment'])
work_int_table

In [ ]:
obs_val = work_int_table.values
alpha = 0.05
test_stat,p,dof,exp_val = chi2_contingency(obs_val)
print('p-value :',p)
print('Test Statistics :',test_stat)

Here p-value is less than 0.05. Hence we reject null hypothesis and we can counclude that Mental health treatment has impact on work interference.

### Statistical test to see if Age has impact on Mental health

In [ ]:
# To test the normality of Age variable
stat, p_value = stats.shapiro(df['Age'])

print('p-value:', p_value)

Here p-value<0.05 (5% Significance level). Null Hypothesis is rejected i.e Alternate Hypothesis is selected.<br>
The data is not normally distributed.

Since data points does not statisfy normality. Mannwhtineyu test is used whic is the equivalent test for unpaired t test.

In [ ]:
MH_yes = df[df['treatment']=='Yes']
MH_no = df[df['treatment']=='No']

In [ ]:
# Graphical understanding:

sns.kdeplot(MH_yes['Age'],label='Under Treatment')
sns.kdeplot(MH_no['Age'],label='Not Under Treatment')
plt.legend()
plt.show()

In [ ]:
# Ho: μ_yes =  μ_no (No difference in age with respect to Mental health issue treatment)
# H1: μ_yes ≠  μ_no (Significant difference in potential with respect to Mental health issue treatment)


test = stats.mannwhitneyu(MH_yes['Age'],MH_no['Age'])
print('p_value:',test[1])

Since p_value < 0.05(5% Significance level). Null Hypothesis is rejected i.e Alternate Hypothesis is selected. 

We can inference that there is Significant difference in age with respect to Mental health issue treatment.



# Impact of mental health on coworkers at workplace

In [ ]:
data=pd.crosstab(df['treatment'],df['obs_consequence'])

In [ ]:

chi=chi2_contingency(data.values)
chi_stats=chi[0]
p_value=chi[1]

In [ ]:
print('test_stats:',chi_stats)
print('pvalue:',p_value)
#here p_value <0.05 , null hypothesis is rejected i.e mental health issue has impact on co workers at work

here p_value <0.05 , null hypothesis is rejected i.e mental health issue has impact on co workers at work

# Is mental health has relation with the family History 

H0: Employees having mental health treatment has no family history of mental illness 

H1: Employees having mental health treatment hasfamily history of mental illness 

In [ ]:
fam_history_table=pd.crosstab(df['treatment'],df['family_history'])
fam_history_table

In [ ]:
test_stats,p_value,dof,expected_values=chi2_contingency(fam_history_table.values)
print('test_stats:',test_stats)
print('pvalue:',p_value)

since p_value <0.05, Null hypothesis is rejected .

Thus, Employees having mental health treatment has family history of mental illness

## MODELLING

In [ ]:
df.head()

In [ ]:
print(df['work_interfere'].unique())
print(df['leave'].unique())
print(df['no_employees'].unique())

In [ ]:
df.head()

In [ ]:
data_cat = df.drop(['work_interfere','leave','treatment','no_employees'],axis=1)
data_ord =  df[['work_interfere','leave','no_employees']]

In [ ]:
encoder = OrdinalEncoder(categories=[['Never','Rarely','Sometimes','Often']])                  
df['work_interfere'] = encoder.fit_transform(df['work_interfere'].values.reshape(-1,1))

encoder = OrdinalEncoder(categories=[["Don't know",'Very difficult','Somewhat difficult','Somewhat easy','Very easy']])
df['leave'] = encoder.fit_transform(df['leave'].values.reshape(-1,1))

encoder = OrdinalEncoder(categories=[['1-5','6-25','26-100', '100-500', '500-1000','More than 1000']])
df['no_employees'] = encoder.fit_transform(df['no_employees'].values.reshape(-1,1))

data_cat_encoding = pd.get_dummies(data_cat,drop_first=True)

In [ ]:
y = df['treatment']

X_final = pd.DataFrame()
X_final = pd.concat([data_cat_encoding,df['work_interfere'],df['leave'],df['no_employees']],axis=1)
X_final

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42)

In [ ]:
import smlr
model = smlr.SMLR(max_iter=1000, tol=1e-8, verbose=5)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
# import xgboost
# from xgboost import XGBClassifier

# xgb = XGBClassifier(learning_rate = 0.01, gamma=2)
# xgb.fit(X_train,y_train)

In [ ]:


print(classification_report(y_test, predictions))

In [ ]:
cf_matrix = confusion_matrix(y_test, predictions)
sns.heatmap(cf_matrix, annot=True,fmt='d', cmap='Purples')
plt.title('Confusion Matrix of SMLR', fontweight='bold', fontsize=16)
plt.xlabel('Predicted', fontweight='bold', fontsize=12)
plt.ylabel('Actual', fontweight='bold', fontsize=12)
plt.show()

In [ ]:
predictions_prob = model.predict_proba(X_test)
roc_auc_score(y_test,predictions_prob[:,1])

In [ ]:
tn=cf_matrix[0,0]
fp=cf_matrix[0,1]
fn=cf_matrix[1,0]
tp=cf_matrix[1,1]

accuracy=(tn+tp)/(tn+tp+fn+fp)
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1_score=2*recall*precision/(recall+precision)
print('Accuracy',accuracy)
print('F1-score',f1_score)
print('Precison',precision)
print('recall',recall)
print('kappa-score',cohen_kappa_score(y_test,predictions))

In [ ]:
features_weight = model.coef_[0]
features_index = [id for id, weight in enumerate(features_weight) if weight!=0]
smlr_features = X_final.iloc[:, features_index].columns.to_list()
print('Top SLMR features: \n', smlr_features)

In [ ]:
from ReliefF import ReliefF

fs = ReliefF(n_neighbors=1, n_features_to_keep=10)
features = fs.fit_transform( X_final.to_numpy(),y.to_numpy()).T

In [ ]:
columns = X_final.columns.to_list()
reliefF_features = []
for feature in features:
    for column in columns:
        idx = (X_final[column] == feature)
        if idx.all() == True:
            reliefF_features.append(column)

print('Top-10 features: \n', reliefF_features)

In [ ]:
feature_names = pd.DataFrame({'features': columns})
feature_names = feature_names.set_index('features')
# feature_names[['smlr', 'reliefF']] = np.nan

In [ ]:
def is_important_feature(row, var):
    names = []
    for index in row.index.to_list():
        #print(index)
        if index in var:
            #print(index,feat)
            names.append(index)
        else:
           # print('no')
            names.append(np.NaN)
    return names


In [ ]:
# feature_names['test']=feature_names.apply(lambda row: is_important_feature(row, smlr_features))
# feature_names

feature_names['SMLR'] = feature_names.apply(lambda row: is_important_feature(row, smlr_features))
feature_names['SMLR']

In [ ]:
feature_names['reliefF'] = feature_names.apply(lambda row: is_important_feature(row, reliefF_features))

In [ ]:
feature_names['counts'] = 5 - feature_names.isnull().sum(axis=1)

In [ ]:
plt.figure(figsize=(20,8))
features_counts = feature_names['counts'].sort_values(ascending=False).to_frame()
ax = sns.barplot(x=features_counts.index, y='counts', data=features_counts,  palette="YlGnBu")
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
ax = ax.set_xticklabels(features_counts.index, rotation = 45, ha="right")

# Modelling-Final 

In [ ]:
score_card = pd.DataFrame(columns=['Model Name','Precision Score', 'Recall Score', 'f1-score','Accuracy Score','roc_auc_score'])
 
def update_scorecard(name,prec,recall,f1,acc,roc):
    global score_card
    score_card = score_card.append({'Model Name':name,'Precision Score': prec,'Recall Score': recall,
                                    'f1-score': f1,'Accuracy Score':acc,'roc_auc_score':roc},ignore_index = True)

In [ ]:
ll=LabelEncoder()
y_label=pd.DataFrame(ll.fit_transform(y))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_label, test_size=0.3, random_state=42)

In [ ]:
update_scorecard('SMLR',0.740740,0.719794,0.7,0.710106,0.770198)

### Decison Tree  

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

In [ ]:
test_pred = dt.predict(X_test)
train_pred = dt.predict(X_train)

print(classification_report(y_train,train_pred))
print(classification_report(y_test,test_pred))

In [ ]:
test_pred_prob = dt.predict_proba(X_test)
roc_auc_score(y_test,test_pred_prob[:,1])

In [ ]:
update_scorecard('Decision Tree',0.68,0.67,0.65,0.64,0.642727)

In [ ]:
tuned_params = [{'criterion': ['entropy', 'gini'],
                'max_depth': range(2,14),
                 'min_samples_split':range(3,10),
                'max_leaf_nodes':range(2,10)}]

                     
dt_grid=GridSearchCV(estimator=DecisionTreeClassifier(random_state=10,),
                     param_grid=tuned_params,cv=5,scoring='f1')
dt_grid.fit(X_train,y_train)

dt_grid.best_params_

In [ ]:
dt_tuned = dt_grid.best_estimator_

In [ ]:
dt_tuned.fit(X_train,y_train)

In [ ]:
test_pred = dt_tuned.predict(X_test)
train_pred = dt_tuned.predict(X_train)

print(classification_report(y_train,train_pred))
print(classification_report(y_test,test_pred))

In [ ]:
test_pred_prob = dt_tuned.predict_log_proba(X_test)
roc_auc_score(y_test,test_pred_prob[:,1])

In [ ]:
update_scorecard('Decision Tree(tuned)',0.75,0.81,0.77,0.75,0.77544)

### BAGGING -RANDOM FOREST

In [ ]:
random_base_model=RandomForestClassifier(random_state=10)
random_base_model=random_base_model.fit(X_train,y_train)

In [ ]:
def con_mat(model,x_test,y_test):
    pred=model.predict(x_test)
    m=confusion_matrix(y_test,pred)
    tn=m[0,0]
    fp=m[0,1]
    fn=m[1,0]
    tp=m[1,1]
    print(sns.heatmap(con_mat(model,x_test,y_test,fmt='d',cbar=False,cmap='Purples')))

In [ ]:
#check model performance

In [ ]:
pred_test=random_base_model.predict(X_test)
pred_train=random_base_model.predict(X_train)

In [ ]:
prob=random_base_model.predict_proba(X_test)

In [ ]:
#confusion metrics

In [ ]:
m=confusion_matrix(y_test,pred_test)
tn=m[0,0]
fp=m[0,1]
fn=m[1,0]
tp=m[1,1]
sns.heatmap(m,annot=True,fmt='d',cmap='Purples',cbar=False)
plt.show()

In [ ]:
accuracy=(tn+tp)/(tn+tp+fn+fp)
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1_score=2*recall*precision/(recall+precision)
print('Accuracy',accuracy)
print('F1-score',f1_score)
print('Precison',precision)
print('recall',recall)
print('kappa-score',cohen_kappa_score(y_test,pred_test))

In [ ]:
#classification report

In [ ]:
print(classification_report(y_train,pred_train))

In [ ]:
print(classification_report(y_test,pred_test))

In [ ]:
roc=roc_auc_score(y_test,prob[:,1])
roc

 From the above output, we can see that there is a difference between the train and test scores; 
thus, we can conclude that the random tree is over-fitted on the train data.

In [ ]:
#lets tune the hyperparameters in the random forest , it helps to avoid the over-fitting of the tree.

In [ ]:
## Tuning the DT model

tuned_params = [{'criterion': ['entropy', 'gini'],
                     'n_estimators': [10, 20, 30, 50, 70, 90],
                     'max_depth': range(2,14,4)}]

                     
rf_grid=GridSearchCV(estimator=RandomForestClassifier(random_state=10,),
                     param_grid=tuned_params,cv=5,scoring='f1')
rf_grid.fit(X_train,y_train)

rf_grid.best_params_

In [ ]:
#lets build the model using tuned params
rf_tune=RandomForestClassifier(n_estimators=50,criterion='entropy',max_depth=10,random_state=10)
rf_tune.fit(X_train,y_train)

In [ ]:
pred_train=rf_tune.predict(X_train)
pred_test=rf_tune.predict(X_test)
#prob_test=rf_tune.predict_proba(X_test)

In [ ]:
print(classification_report(y_train,pred_train))

In [ ]:
print(classification_report(y_test,pred_test))

In [ ]:
#lets choose optimal value of threshold
prob_test=rf_tune.predict_proba(X_test)
#prob_test[:,1]

In [ ]:

fpr,tpr,th=roc_curve(y_test,prob_test[:,1])
df_roc=pd.DataFrame({'fpr':fpr,'tpr':tpr,'thresh':th})
df_roc['youden']=df_roc['tpr']-df_roc['fpr']
df_roc.sort_values(by='youden',ascending=False)
#use thresh as 0.498

In [ ]:
pred_test_roc=[0 if i <0.498 else 1 for i in prob_test[:,1]]
print(classification_report(y_test,pred_test_roc))

In [ ]:
roc_auc_score(y_test,prob_test[:,1])

In [ ]:
# feature importance

In [ ]:
df_rf_imp=pd.DataFrame({'feature':X_train.columns,'importance':rf_tune.feature_importances_})
df_rf_imp=df_rf_imp.sort_values(by='importance',ascending=False)

In [ ]:
plt.figure(figsize=(16,7))
sns.barplot(df_rf_imp['feature'],df_rf_imp['importance'])
plt.xticks(rotation=90);

In [ ]:
update_scorecard('random_base_model',precision,recall,f1_score,accuracy,roc)
update_scorecard('random_tune_model',0.72,0.72,0.72,0.72,0.790)
score_card

### BOOSTING TECHNIQUE - ADABOOSTING

In [ ]:
ada_base_model=AdaBoostClassifier(random_state=10)
ada_base_model.fit(X_train,y_train)

In [ ]:
pred_train=ada_base_model.predict(X_train)
pred_test=ada_base_model.predict(X_test)
#prob_test=rf_tune.predict_proba(X_test)

In [ ]:
m=confusion_matrix(y_test,pred_test)
tn=m[0,0]
fp=m[0,1]
fn=m[1,0]
tp=m[1,1]
sns.heatmap(m,annot=True,fmt='d',cbar=False,cmap='Purples')
plt.show()

In [ ]:

accuracy=(tn+tp)/(tn+tp+fn+fp)
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1_score=2*recall*precision/(recall+precision)
print('Accuracy',accuracy)
print('F1-score',f1_score)
print('Precison',precision)
print('recall',recall)
print('kappa-score',cohen_kappa_score(y_test,pred_test))

In [ ]:
print(classification_report(y_train,pred_train))

In [ ]:
print(classification_report(y_test,pred_test))

In [ ]:
prob_test=ada_base_model.predict_proba(X_test)
roc=roc_auc_score(y_test,prob_test[:,1])

In [ ]:
# the model is performing well on both train and test data set  but it can be further improved 

In [ ]:
update_scorecard('ada_base_model',precision,recall,f1_score,accuracy,roc)
score_card

In [ ]:
# lets tune model
params=[{'n_estimators': [10, 20, 30, 50, 70, 90],
         'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
        }]
ada_grid=GridSearchCV(estimator=AdaBoostClassifier(random_state=10),param_grid=params,cv=3,scoring='f1')
ada_grid.fit(X_train,y_train)
ada_grid.best_params_

In [ ]:
# build model using tuned params
ada_tune=AdaBoostClassifier(learning_rate=0.3,n_estimators=10,random_state=10)
ada_tune.fit(X_train,y_train)

In [ ]:
pred_train=ada_tune.predict(X_train)
pred_test=ada_tune.predict(X_test)

In [ ]:
print(classification_report(y_train,pred_train))

In [ ]:
print(classification_report(y_test,pred_test))

In [ ]:
#lets choose optimal value of threshold
prob_test=ada_tune.predict_proba(X_test)
#prob_test[:,1]
fpr,tpr,th=roc_curve(y_test,prob_test[:,1])
df_roc_ada=pd.DataFrame({'fpr':fpr,'tpr':tpr,'thresh':th})
df_roc_ada['youden']=df_roc_ada['tpr']-df_roc_ada['fpr']
df_roc_ada.sort_values(by='youden',ascending=False).head(5)
#use thresh as 0.517

In [ ]:
pred_test_roc=[0 if i <0.517 else 1 for i in prob_test[:,1]]
print(classification_report(y_test,pred_test_roc))

In [ ]:
roc_auc_score(y_test,prob_test[:,1])

In [ ]:
update_scorecard('ada_tune_model',0.75,0.75,0.75,0.75,0.7952)
score_card

In [ ]:
df_ada_imp=pd.DataFrame({'feature':X_train.columns,'importance':ada_tune.feature_importances_})
df_ada_imp=df_ada_imp.sort_values(by='importance',ascending=False)


In [ ]:
plt.figure(figsize=(16,7))
sns.barplot(df_ada_imp['feature'],df_ada_imp['importance'])
plt.xticks(rotation=90);

In [ ]:
X_train_1 = X_train.drop(['work_interfere','family_history_Yes','care_options_Yes','benefits_Yes'],axis=1)
X_test_1 = X_test.drop(['work_interfere','family_history_Yes','care_options_Yes','benefits_Yes'],axis=1)

In [ ]:
# build model to see impact of other features 
ada_feat=AdaBoostClassifier(learning_rate=0.3,n_estimators=10,random_state=10)
ada_feat.fit(X_train_1,y_train)

In [ ]:
pred_train=ada_feat.predict(X_train_1)
pred_test=ada_feat.predict(X_test_1)

print(classification_report(y_train,pred_train))

In [ ]:
df_ada_feat=pd.DataFrame({'feature':X_train_1.columns,'importance':ada_feat.feature_importances_})
df_ada_feat=df_ada_feat.sort_values(by='importance',ascending=False)


In [ ]:
plt.figure(figsize=(16,7))
sns.barplot(df_ada_feat['feature'],df_ada_feat['importance'])
plt.xticks(rotation=90);

In [ ]:
#lets build model using imp features

In [ ]:
sig_ada=df_ada_imp[df_ada_imp['importance']>0]['feature']
sig_ada

In [ ]:
x_train_ada=X_train[sig_ada]
x_test_ada=X_test[sig_ada]

In [ ]:
ada_sig_model=AdaBoostClassifier(learning_rate=0.3,n_estimators=10,random_state=10)
ada_sig_model.fit(x_train_ada,y_train)


In [ ]:
pred_test=ada_sig_model.predict(x_test_ada)

In [ ]:
print(classification_report(y_test,pred_test))

In [ ]:
prob=ada_sig_model.predict_proba(x_test_ada)
roc_auc_score(y_test,prob[:,1])

In [ ]:
update_scorecard('ada_significant_feature_model',0.75,0.75,0.75,0.75,0.7952)
score_card

### XGBOOST

In [ ]:
xgboost_base_model=XGBClassifier(eval_metric='error')
xgboost_base_model.fit(X_train,y_train)

In [ ]:
pred_train=xgboost_base_model.predict(X_train)
pred_test=xgboost_base_model.predict(X_test)

In [ ]:
m=confusion_matrix(y_test,pred_test)
tn=m[0,0]
fp=m[0,1]
fn=m[1,0]
tp=m[1,1]
sns.heatmap(m,annot=True,cmap='Purples',cbar=False,fmt='d')
plt.show()

In [ ]:
accuracy=(tn+tp)/(tn+tp+fn+fp)
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1_score=(2*recall*precision)/(recall+precision)
print('Accuracy',accuracy)
print('F1-score',f1_score)
print('Precison',precision)
print('recall',recall)
print('kappa-score',cohen_kappa_score(y_test,pred_test))

In [ ]:
print(classification_report(y_train,pred_train))

In [ ]:
print(classification_report(y_test,pred_test))

From the above output, we can see that there is a difference between the train and test scores; thus, we can conclude that the random tree is over-fitted on the train data.

In [ ]:
prob=xgboost_base_model.predict_proba(X_test)
roc=roc_auc_score(y_test,prob[:,1])
roc

In [ ]:
update_scorecard('xgboost_base_model',precision,recall,f1_score,accuracy,roc)
score_card

In [ ]:
# lets tune the model to find optimal parameters for better performance

In [ ]:
tuning_params = {'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
                     'max_depth': range(3,10),
                     'gamma': [0, 1, 2, 3, 4]}

xgb_grid = GridSearchCV(estimator = XGBClassifier(eval_metric='error'), param_grid = tuning_params, cv = 3, scoring = 'roc_auc').fit(X_train, y_train)

xgb_grid.best_params_

In [ ]:
#model build using tuned params
xgb_tuned = XGBClassifier(learning_rate = xgb_grid.best_params_.get('learning_rate'),
                max_depth = xgb_grid.best_params_.get('max_depth'),
                gamma = xgb_grid.best_params_.get('gamma')).fit(X_train, y_train)

ypred_train_tuned = xgb_tuned.predict(X_train)
ypred_tuned = xgb_tuned.predict(X_test)
ypred_prob_tuned =xgb_tuned.predict_proba(X_test)[:,1]

In [ ]:
# Train report
print(classification_report(y_train,ypred_train_tuned))

In [ ]:
# Test report
print(classification_report(y_test,ypred_tuned))

In [ ]:
prob=xgb_tuned.predict_proba(X_test)
roc_auc_score(y_test,prob[:,1])

In [ ]:
update_scorecard('xgb_tuned_model',0.74,0.74,0.74,0.74,0.781)
score_card

In [ ]:
important_features = pd.DataFrame({'Features': X_train.columns, 
                                   'Importance': xgb_tuned.feature_importances_})
important_features = important_features.sort_values('Importance', ascending = False)

plt.figure(figsize=(14,7))
sns.barplot(y = 'Importance', x = 'Features', data = important_features)
plt.title('Feature Importance', fontsize = 15)
plt.xlabel('Importance', fontsize = 15)
plt.ylabel('Features', fontsize = 15)
plt.xticks(rotation=90)
plt.show()

### Stacking

In [ ]:
# consider the various algorithms as base learners
base_learners = [('rf', RandomForestClassifier(n_estimators=50,criterion='entropy',max_depth=10,random_state=10)),
                 ('ada',AdaBoostClassifier(learning_rate=0.3,n_estimators=10,random_state=10)),
                 ('xgb', XGBClassifier(learning_rate = 0.5,max_depth = 3,gamma = 4,eval_metric='error'))]

final = KNeighborsClassifier(metric='euclidean',n_neighbors=7)

stack_model = StackingClassifier(estimators = base_learners, final_estimator = final).fit(X_train, y_train)

ypred_train_tuned = stack_model.predict(X_train)
ypred_tuned = stack_model.predict(X_test)
ypred_prob_tuned =stack_model.predict_proba(X_test)[:,1]

In [ ]:
# Train report
print(classification_report(y_train,ypred_train_tuned))

In [ ]:
# Test report
print(classification_report(y_test,ypred_tuned))

In [ ]:
prob=stack_model.predict_proba(X_test)
roc=roc_auc_score(y_test,prob[:,1])

In [ ]:
update_scorecard('Stacked(RF,ada,XG)',0.68,0.68,0.68,0.68,roc)

In [ ]:
# consider the various algorithms as base learners
base_learners = [('rf', RandomForestClassifier(n_estimators=50,criterion='entropy',max_depth=10,random_state=10)),
                 ('ada',AdaBoostClassifier(learning_rate=0.3,n_estimators=10,random_state=10)),
                 ('smlr', smlr.SMLR()),
                 ('xgb', XGBClassifier(learning_rate = 0.5,max_depth = 3,gamma = 4,eval_metric='error'))]

final = KNeighborsClassifier(metric='euclidean',n_neighbors=7)

stack_model = StackingClassifier(estimators = base_learners, final_estimator = final).fit(X_train, y_train)

In [ ]:
ypred_train_tuned = stack_model.predict(X_train)
ypred_tuned = stack_model.predict(X_test)
ypred_prob_tuned =stack_model.predict_proba(X_test)[:,1]

# Train report
print(classification_report(y_train,ypred_train_tuned))

# Test report
print(classification_report(y_test,ypred_tuned))

In [ ]:
prob=stack_model.predict_proba(X_test)
roc=roc_auc_score(y_test,prob[:,1])

In [ ]:
update_scorecard('Stacked(RF,ada,SMLR,XG)',0.77,0.68,0.72,0.72,roc)

In [ ]:
# consider the various algorithms as base learners
base_learners = [('rf', RandomForestClassifier(n_estimators=50,criterion='entropy',max_depth=10,random_state=10)),
                 #('ada',AdaBoostClassifier(learning_rate=0.3,n_estimators=10,random_state=10)),
                 ('smlr', smlr.SMLR()),
                 ('xgb', XGBClassifier(learning_rate = 0.5,max_depth = 3,gamma = 4,eval_metric='error'))]

final = KNeighborsClassifier(metric='euclidean',n_neighbors=7)

stack_model = StackingClassifier(estimators = base_learners, final_estimator = final).fit(X_train, y_train)

In [ ]:
ypred_train_tuned = stack_model.predict(X_train)
ypred_tuned = stack_model.predict(X_test)
ypred_prob_tuned =stack_model.predict_proba(X_test)[:,1]

# Train report
print(classification_report(y_train,ypred_train_tuned))

# Test report
print(classification_report(y_test,ypred_tuned))

In [ ]:
stack_model.feature_names_in_

In [ ]:
stack_model.n_features_in_

In [ ]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred_test  =  classifier.predict(X_test)
y_pred_train  =  classifier.predict(X_train)
print(classification_report(y_train,y_pred_train))
print(classification_report(y_test,y_pred_test))

In [ ]:
clf = SVC(kernel='linear', C = 1.0,)
clf.fit(X_train,y_train)

In [ ]:
y_pred_test  =  clf.predict(X_test)
y_pred_train  =  clf.predict(X_train)
print(classification_report(y_train,y_pred_train))
print(classification_report(y_test,y_pred_test))

In [ ]:
SVC().get_params().keys()

### Random-forest with relief

In [ ]:
# selected important features using relief
X_train_rf = X_train[reliefF_features]
X_test_rf = X_test[reliefF_features]

In [ ]:
random_feat=RandomForestClassifier(random_state=10)
random_feat=random_base_model.fit(X_train_rf,y_train)

In [ ]:
#check model performance

pred_test=random_base_model.predict(X_test_rf)
pred_train=random_base_model.predict(X_train_rf)

prob=random_base_model.predict_proba(X_test_rf)

In [ ]:
#confusion metrics

m=confusion_matrix(y_test,pred_test)
tn=m[0,0]
fp=m[0,1]
fn=m[1,0]
tp=m[1,1]
sns.heatmap(m,annot=True,fmt='d',cmap='Purples',cbar=False)
plt.show()

In [ ]:
accuracy=(tn+tp)/(tn+tp+fn+fp)
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1_score=2*recall*precision/(recall+precision)
print('Accuracy',accuracy)
print('F1-score',f1_score)
print('Precison',precision)
print('recall',recall)
print('kappa-score',cohen_kappa_score(y_test,pred_test))

In [ ]:
#classification report

print(classification_report(y_train,pred_train))

print(classification_report(y_test,pred_test))

roc=roc_auc_score(y_test,prob[:,1])
roc

In [ ]:
update_scorecard('random_with_reliefF',0.74419,0.64,0.688,0.691489,0.711491)
score_card

### Ada with reliefF features

In [ ]:
ada_base_model=AdaBoostClassifier(random_state=10)
ada_base_model.fit(X_train_rf,y_train)

In [ ]:
pred_train=ada_base_model.predict(X_train_rf)
pred_test=ada_base_model.predict(X_test_rf)
#prob_test=rf_tune.predict_proba(X_test)

In [ ]:
m=confusion_matrix(y_test,pred_test)
tn=m[0,0]
fp=m[0,1]
fn=m[1,0]
tp=m[1,1]
sns.heatmap(m,annot=True,fmt='d',cbar=False,cmap='Purples')
plt.show()

In [ ]:
accuracy=(tn+tp)/(tn+tp+fn+fp)
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1_score=2*recall*precision/(recall+precision)
print('Accuracy',accuracy)
print('F1-score',f1_score)
print('Precison',precision)
print('recall',recall)
print('kappa-score',cohen_kappa_score(y_test,pred_test))

In [ ]:
print(classification_report(y_train,pred_train))

print(classification_report(y_test,pred_test))

prob_test=ada_base_model.predict_proba(X_test_rf)
roc=roc_auc_score(y_test,prob_test[:,1])
roc

In [ ]:
update_scorecard('ada_with_reliefF',precision,recall,f1_score,accuracy,roc)

In [ ]:
# consider the various algorithms as base learners
base_learners = [('rf', RandomForestClassifier(n_estimators=50,criterion='entropy',max_depth=10,random_state=10)),
                 ('dt',DecisionTreeClassifier(max_depth=4, max_leaf_nodes=6, min_samples_split=3,random_state=10)),
                 ('smlr', smlr.SMLR()),
                 ('xgb', XGBClassifier(learning_rate = 0.5,max_depth = 3,gamma = 4,eval_metric='error'))]

final = KNeighborsClassifier(metric='euclidean',n_neighbors=7)

stack_model_dt = StackingClassifier(estimators = base_learners, final_estimator = final).fit(X_train, y_train)

ypred_train_tuned = stack_model.predict(X_train)
ypred_tuned = stack_model.predict(X_test)
ypred_prob_tuned =stack_model.predict_proba(X_test)[:,1]

# Train report
print(classification_report(y_train,ypred_train_tuned))

# Test report
print(classification_report(y_test,ypred_tuned))

### Compare Result

In [ ]:
score_card

In [ ]:
# Of all models, Adaboosting model has better performance metrics than other models.
# But while looking at the importance of features, it is only considering few features.
# Even after removing the highly affecting features, it only gives importance to few features. 
# Hence it is not capturing the dataset completely.

# So we select stacked model WITH SMLR, RANDOM FOREST AND XGBOOST. FINAL-KNN as final model. 
# The model performace is good and is considering many features

### Final Model

### STACKED MODEL WITH SMLR, RANDOM FOREST AND XGBOOST. FINAL-KNN

In [ ]:
# consider the various algorithms as base learners
base_learners = [('rf', RandomForestClassifier(n_estimators=50,criterion='entropy',max_depth=10,random_state=10)),
                 #('ada',AdaBoostClassifier(learning_rate=0.3,n_estimators=10,random_state=10)),
                 ('smlr', smlr.SMLR()),
                 ('xgb', XGBClassifier(learning_rate = 0.5,max_depth = 3,gamma = 4,eval_metric='error'))]

final = KNeighborsClassifier(metric='euclidean',n_neighbors=7)

stack_model_final = StackingClassifier(estimators = base_learners, final_estimator = final).fit(X_train, y_train)

In [ ]:
ypred_train_tuned = stack_model_final.predict(X_train)
ypred_tuned = stack_model_final.predict(X_test)
ypred_prob_tuned =stack_model_final.predict_proba(X_test)[:,1]

# Train report
print(classification_report(y_train,ypred_train_tuned))

# Test report
print(classification_report(y_test,ypred_tuned))



In [ ]:
m=confusion_matrix(y_test,ypred_tuned)
tn=m[0,0]
fp=m[0,1]
fn=m[1,0]
tp=m[1,1]
sns.heatmap(m,annot=True,fmt='d',cbar=False,cmap='Purples')
plt.show()

In [ ]:
accuracy=(tn+tp)/(tn+tp+fn+fp)
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1_score=2*recall*precision/(recall+precision)
print('Accuracy',accuracy)
print('F1-score',f1_score)
print('Precison',precision)
print('recall',recall)
print('kappa-score',cohen_kappa_score(y_test,ypred_tuned))

In [ ]:
prob=stack_model_final.predict_proba(X_test)
roc=roc_auc_score(y_test,prob[:,1])
print('roc score:',roc)

In [ ]:
fpr,tpr,th=roc_curve(y_test,prob[:,1])

plt.figure(figsize=(10,6))
plt.plot(fpr,tpr)
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.plot([0,1],[0,1],'r--')
plt.title('ROC Curve for stacked RF,SMLR,XGB',fontsize=12)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid('True')
plt.show()

In [ ]:
print('No of features selected in training the model :',stack_model.n_features_in_)
stack_model.feature_names_in_

In [ ]:
update_scorecard('Stacked (RF,SMLR,XG)',0.777143,0.68,0.725333,0.726064,roc)

From the above importance’s of RF & XG, we may say that the employees who have faced work interference and have a family history of mental illness are prone to have mental health issue. We can also say that with age the mental mealth is affected.

## Bootstrapping 

In [ ]:
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

In [ ]:
# restructure data for input into model
X_train_bs = X_train.values.reshape((len(X_train.values), 35))
y_train_bs = y_train.values.ravel()
X_test_bs = X_test.values.reshape((len(X_test.values), 35))
y_test_bs = y_test.values.ravel()


In [ ]:
# train model
base_learners = [('rf', RandomForestClassifier(n_estimators=50,criterion='entropy',max_depth=10,random_state=10)),
                 ('smlr', smlr.SMLR()),
                 ('xgb', XGBClassifier(learning_rate = 0.5,max_depth = 3,gamma = 4,eval_metric='error'))]

final = KNeighborsClassifier(metric='euclidean',n_neighbors=7)

stack_model_final = StackingClassifier(estimators = base_learners, final_estimator = final).fit(X_train, y_train)


# bootstrap predictions
accuracy_stack = []
n_iterations = 1000
for i in range(n_iterations):
    X_bs, y_bs = resample(X_train_bs, y_train_bs, replace=True)
    # make predictions
    y_hat = stack_model_final.predict(X_bs)
    # evaluate model
    score = accuracy_score(y_bs, y_hat)
    accuracy_stack.append(score)
    
print(np.mean(accuracy))

In [ ]:
# get median
median_stack = np.percentile(accuracy_stack, 50)

# get 95% interval
alpha = 100-95
lower_ci = np.percentile(accuracy_stack, alpha/2)
upper_ci = np.percentile(accuracy_stack, 100-alpha/2)

print(f"Model accuracy is reported on the test set. 1000 bootstrapped samples " 
      f"were used to calculate 95% confidence intervals.\n"
      f"Median accuracy is {median_stack:.2f} with a 95% a confidence "
      f"interval of [{lower_ci:.2f},{upper_ci:.2f}].")


Decision criteria:<br>
While hyper tuning the parameters, we focus on reducing False Negative i.e, the employee is suffering from mental illness and the model predicts him/her as healthy. This will be a loss employee. At the same time the accuracy of the model should be high.  